In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt

from tensorflow import keras

from keras import layers
from keras import losses

from keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import TimeseriesGenerator


data = pd.read_csv('/content/Final_eskom_vector_grid_health.csv')
data = pd.DataFrame(data)
data['Time'] = pd.to_datetime(data['DateTime'])
data.set_index('Time', inplace=True)


Y_columns = [
    'ILS Usage',
    'MLR', 'IOS Excl ILS and MLR',
     'Total UCLF+OCLF'
]
Y = data[Y_columns].values
X = data.drop(columns=Y_columns, axis=1)
X.drop(['DateTime'], axis =1, inplace = True)
X = X.values

X_train_temp, X_test, Y_train_temp, Y_test = train_test_split(X, Y, test_size=0.10, shuffle=False, random_state=42)

# Then, split the temporary test set into validation (50%) and final test (50%)
X_train, X_val, Y_train, Y_val = train_test_split(X_train_temp, Y_train_temp, test_size=0.10, shuffle=False, random_state=42)

# Print the shapes of the resulting sets
print("Training set (X_train, Y_train) shapes:", X_train.shape, Y_train.shape)
print("Validation set (X_val, Y_val) shapes:", X_val.shape, Y_val.shape)
print("Test set (X_test, Y_test) shapes:", X_test.shape, Y_test.shape)


scaler = StandardScaler()
X_train_normalised = scaler.fit_transform(X_train)
X_test_normalised = scaler.transform(X_test)
X_val_norm = scaler.transform(X_val)

Y_train_normalised = scaler.fit_transform(Y_train)
Y_test_normalised = scaler.transform(Y_test)
Y_val_norm = scaler.transform(Y_val)


timesteps = 24*7
window_length = timesteps
batch_size = 16
num_features = 15
train_generator = TimeseriesGenerator(X_train_normalised, Y_train_normalised,length=window_length, batch_size=batch_size)
test_generator = TimeseriesGenerator(X_test_normalised, Y_test_normalised,length=window_length, batch_size=batch_size)
val_generator = TimeseriesGenerator(X_val_norm, Y_val_norm, length=window_length, batch_size=batch_size)


Training set (X_train, Y_train) shapes: (23899, 15) (23899, 4)
Validation set (X_val, Y_val) shapes: (2656, 15) (2656, 4)
Test set (X_test, Y_test) shapes: (2951, 15) (2951, 4)


## Final test on architecture type 2Layers-1B-1L vs 3Layers-2B-1L vs 2Layers-2L

In [3]:
epochs = 50
num_layers = 16
patience = 8
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, mode='min', verbose=True, restore_best_weights=True)

In [ ]:
## 2 layers 2 LSTM
 #num epochs without improvement

model_simple = tf.keras.Sequential([
  layers.LSTM(16, input_shape = (window_length, 15), return_sequences=True, activation='tanh'),
  layers.Dropout(0.2),
  layers.LSTM(16, input_shape = (window_length, 15)),
  layers.Dropout(0.2),
  layers.Dense(4, activation='linear')])

model_simple.build((None, timesteps, 15))
model_simple.summary()

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, mode='min', verbose=True, restore_best_weights=True)


model_simple.compile(loss=tf.keras.losses.Huber(delta = 0.3),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=[tf.keras.losses.Huber()])
history_simple = model_simple.fit(train_generator, epochs=epochs, validation_data= val_generator,  callbacks=[early_stopping], verbose=1)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 168, 16)           2048      
                                                                 
 dropout (Dropout)           (None, 168, 16)           0         
                                                                 
 lstm_1 (LSTM)               (None, 16)                2112      
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense (Dense)               (None, 4)                 68        
                                                                 
Total params: 4228 (16.52 KB)
Trainable params: 4228 (16.52 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/50
1484/1484 [

In [ ]:
## 2 layers 1 Birectional 1 LSTM

modelH03 = tf.keras.Sequential([
  layers.Bidirectional(
  layers.LSTM(16, input_shape = (window_length, 15), return_sequences=True, activation='tanh')),
  layers.Dropout(0.2),
  layers.LSTM(16,  return_sequences=False),
  layers.Dropout(0.2),
  layers.Dense(4, activation='linear')])

modelH03.build((None, timesteps, 15))
modelH03.summary()

modelH03.compile(loss=tf.keras.losses.Huber(delta=0.3),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=[tf.keras.losses.Huber()])

history_2Layers_simple = modelH03.fit(train_generator, epochs=epochs, validation_data= val_generator,  callbacks=[early_stopping], verbose=1)


In [ ]:
## 3 layers 2 Bidirectional 1 LSTM
model_decay_3 = tf.keras.Sequential([
  layers.Bidirectional(
  layers.LSTM(16, input_shape = (window_length, 15), return_sequences=True, activation='tanh')),
  layers.Dropout(0.2),
  layers.Bidirectional(layers.LSTM(16,  return_sequences=True)),
  layers.Dropout(0.2),
  layers.LSTM(16,  return_sequences=False),
  layers.Dropout(0.2),
  layers.Dense(4, activation='linear')])

model_decay_3.build((None, timesteps, 15))
model_decay_3.summary()

model_decay_3.compile(loss=tf.keras.losses.Huber(delta=0.3),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=[tf.keras.losses.Huber()])
model_history_decay_3 = model_decay_3.fit(train_generator, epochs=epochs, validation_data= val_generator,  callbacks=[early_stopping], verbose=1)

In [ ]:
history_dicts = [history_simple.history, history_2Layers_simple.history, model_history_decay_3.history]

# Define labels for each model
model_labels = ['Model 1', 'Model 2', 'Model 3']

# Create a new figure
plt.figure(figsize=(10, 6))

# Loop through each model's history
for i, history_dict in enumerate(history_dicts):
    error = history_dict['huber_loss']
    val_error = history_dict['val_huber_loss']
    loss = history_dict['loss']
    val_loss = history_dict['val_loss']

    epochs = range(1, len(error) + 1)

    # Plot training loss with dots
    plt.plot(epochs, loss, 'o-', label=f'{model_labels[i]} - Training loss')

    # Plot validation loss with lines
    plt.plot(epochs, val_loss, '-', label=f'{model_labels[i]} - Validation loss')

plt.title('Training and validation loss for Multiple Models')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.show()